In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
from unittest.mock import patch
from openai_adapter import OpenAIAdapter


adapter = OpenAIAdapter()
question = "たこしあって何？検索して教えて"
answer = adapter.create_chat(question)
print("返答:", answer)
print("チャットログ:", adapter.chat_log)


返答: たこしあ？うーん、よくわからないな。調べてみて！
チャットログ: [{'question': 'たこしあって何？検索して教えて', 'answer': 'たこしあ？うーん、よくわからないな。調べてみて！'}]


### langchainで回答 

In [2]:
from langchain_openai import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()  # .envファイルから環境変数を読み込む

True

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key= os.getenv("OPENAI_API_KEY"),  # if you prefer to pass api key in directly instaed of using env vars
    output_version="responses/v1"
    # base_url="...",
    # organization="...",
    # other params...
)

tool = {"type": "web_search_preview"}
llm_with_tools = llm.bind_tools([tool])

response = llm_with_tools.invoke("たこしあって何？検索して教えて")

Response: {'id': 'ws_68624b668c7481a08ea54aeba7d793300af2a8b48f96685b', 'action': {'query': 'たこしあ', 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}


In [10]:
print("Response:", response.content)

Response: [{'id': 'ws_68624b668c7481a08ea54aeba7d793300af2a8b48f96685b', 'action': {'query': 'たこしあ', 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}, {'type': 'text', 'text': '「たこしあ」は、ゲーム『モンスターハンター：ワイルズ』の開発者インタビューで紹介されたシチュエーションから派生したネットスラングです。インタビューでは、ハンターが目標でない大型モンスターを偶然発見し、傷ついたそのモンスターを追いかける場面が描かれています。このシチュエーションを指す言葉として「たこしあ」が使われるようになりました。 ([detail.chiebukuro.yahoo.co.jp](https://detail.chiebukuro.yahoo.co.jp/qa/question_detail/q13316701803?utm_source=openai))\n\nしかし、実際のゲームプレイでは、このような偶然の遭遇は稀であり、開発者が意図したほど頻繁には発生しないとの報告もあります。 ([kurashilatte.hatenablog.jp](https://kurashilatte.hatenablog.jp/entry/2025/06/25/132703?utm_source=openai))\n\nこの言葉は、プレイヤー間でのユーモアや共感を生むために使われ、SNSやゲーム掲示板で広まりました。例えば、X（旧Twitter）では「#たこシあ」などのハッシュタグが使用され、プレイヤー同士のコミュニケーションの一環として楽しまれています。 ([kurashilatte.hatenablog.jp](https://kurashilatte.hatenablog.jp/entry/2025/06/25/132703?utm_source=openai)) ', 'annotations': [{'end_index': 289, 'start_index': 167, 'title': 'モンハンワイルズ「たこしあ」ってなんですか？ - 「たこシあ」とは... -

In [11]:
# 本文だけを抽出
text_part = next(item["text"] for item in response.content if item.get("type") == "text")

# 表示
print("本文のみ:", text_part)


本文のみ: 「たこしあ」は、ゲーム『モンスターハンター：ワイルズ』の開発者インタビューで紹介されたシチュエーションから派生したネットスラングです。インタビューでは、ハンターが目標でない大型モンスターを偶然発見し、傷ついたそのモンスターを追いかける場面が描かれています。このシチュエーションを指す言葉として「たこしあ」が使われるようになりました。 ([detail.chiebukuro.yahoo.co.jp](https://detail.chiebukuro.yahoo.co.jp/qa/question_detail/q13316701803?utm_source=openai))

しかし、実際のゲームプレイでは、このような偶然の遭遇は稀であり、開発者が意図したほど頻繁には発生しないとの報告もあります。 ([kurashilatte.hatenablog.jp](https://kurashilatte.hatenablog.jp/entry/2025/06/25/132703?utm_source=openai))

この言葉は、プレイヤー間でのユーモアや共感を生むために使われ、SNSやゲーム掲示板で広まりました。例えば、X（旧Twitter）では「#たこシあ」などのハッシュタグが使用され、プレイヤー同士のコミュニケーションの一環として楽しまれています。 ([kurashilatte.hatenablog.jp](https://kurashilatte.hatenablog.jp/entry/2025/06/25/132703?utm_source=openai)) 


In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
import os

# OpenAI APIキー
api_key = os.getenv("OPENAI_API_KEY")

# LLM
llm = ChatOpenAI(
    model="gpt-4-1106-preview",
    api_key=api_key,
    temperature=0
)

# 疑似的なWeb検索ツール
def dummy_search(query: str) -> str:
    return f"『{query}』について検索しました。（これはダミー結果です）"

tool = Tool.from_function(
    func=dummy_search,
    name="web_search_preview",
    description="ユーザーの質問に対してWeb検索っぽい返答を返すツール"
)

# ツールをLLMにバインド
llm_with_tools = llm.bind_tools([tool])

# 実行
response = llm_with_tools.invoke("今日の良いニュースは？")
print(response.content)


KeyboardInterrupt: 

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

def create_chat(self, question):

    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        api_key=os.getenv("OPENAI_API_KEY"),
        output_version="responses/v1"
    )

    tool = {"type": "web_search_preview"}
    llm_with_tools = llm.bind_tools([tool])

    # メッセージリストを構築
    messages = [SystemMessage(content=self.system_prompt)]
    for c in self.chat_log:
        messages.append(HumanMessage(content=c["question"]))
        messages.append(AIMessage(content=c["answer"]))
    messages.append(HumanMessage(content=question))

    # LLMにメッセージリストを渡して回答を取得
    response = llm_with_tools.invoke(messages)
    answer = next(item["text"] for item in response.content if item.get("type") == "text")
    self._update_messages(question, answer)
    return answer

In [16]:
print(create_chat(adapter, "たこしあって何？検索して教えてssss"))

「たこしあ」は、モンスターハンターシリーズの最新作『モンスターハンター ワイルズ』において、プレイヤー間で使われる造語です。この言葉は、ゲーム内での特定の状況や行動を指すもので、主に以下のような意味合いで使用されます。

**1. ゲーム内での特定の行動や状況を指す**

「たこしあ」は、ゲーム内での特定の行動や状況を指す言葉として使われます。具体的には、モンスターとの戦闘中に仲間が倒されてしまった際や、クエストで失敗した際に、軽い冗談や皮肉として使用されることが多いです。この言葉を使うことで、プレイヤー間のコミュニケーションが活発になり、ゲームの楽しさを増す役割を果たしています。 ([life.awaisora.com](https://life.awaisora.com/2025/06/28/dd631ee7-f76c-487b-b211-edf94b046b32/?utm_source=openai))

**2. プレイヤー間のコミュニケーションの一環として使用**

また、「たこしあ」は、プレイヤー間のコミュニケーションの一環として使用されることもあります。例えば、モンスターが予測できない動きをしたり、特定のイベントが発生した時に「たこしあ！」と叫ぶことで、仲間との共感を深めることができます。このように、ゲーム内でのストレス発散や楽しさを共有するための手段としても活用されています。 ([life.awaisora.com](https://life.awaisora.com/2025/06/26/e9811ab5-3262-4c65-8744-e4bc94da29fa/?utm_source=openai))

**3. 他のオンラインゲームでも類似の表現が存在**

「たこしあ」のような表現は、モンハンワイルズに限らず、他のオンラインゲームにおいても見られます。ゲーム内でのコミュニケーションや、仲間同士の絆を深めるために使われることが多く、ポジティブな意味合いも含まれることがあります。このような言葉の使用は、ゲーム内での挑戦や失敗を受け入れるためのユーモアとしても捉えることができます。 ([life.awaisora.com](https://life.awaisora.com/2025/06/28/dd631ee7-f76c-487b-b211-edf